#  RAG 체인 구성

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

# 다양한 문서 형식 처리하기

- 역할: Document Loader는 다양한 소스에서 문서를 로드
- 구현: 
    - Document Loader는 BaseLoader 인터페이스를 통해서 구현
    - `.load()` 또는 `.lazy_load()` 메서드를 통해 동일한 방식으로 호출 
    - 대용량 데이터셋의 경우 메모리 효율을 위해 `.lazy_load()`를 사용하는 것을 권장 
- 종류:
    - PDF 파일 로더
    - 웹 페이지 로더 
    - CSV 데이터 로더
    - 디렉토리 로더
    - HTML 데이터 로더
    - JSON 데이터 로더
    - Markdown 데이터 로더
    - Microsoft Office 데이터 로더

### 1. **웹 문서 로더**

- uv add bs4 langchain_community

In [3]:
from langchain_community.document_loaders import WebBaseLoader

# 기본적인 텍스트 추출
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/", 
        "https://js.langchain.com/",
        ],
    )

# 동기 로딩
web_docs = web_loader.load()

len(web_docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


2

In [7]:
web_docs[0].metadata

{'source': 'https://python.langchain.com/',
 'title': 'Introduction | 🦜️🔗 LangChain',
 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).',
 'language': 'en'}

In [5]:
print(web_docs[0].page_content)






Introduction | 🦜️🔗 LangChain






Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to stream chat model responsesHow to add default invocation args

### 2. **CSV 파일 로더**

In [8]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 파일 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv", encoding="utf-8")
csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

문서의 수: 10
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': './data/kbo_teams_2023.csv', 'row': 0}
--------------------------------------------------
처음 문서의 내용: 
 Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


In [13]:
## 소스 컬럼 지정

csv_loader = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    encoding="utf-8",
    source_column="Team"  # 이 컬럼의 값이 메타데이터의 source로 사용됨
)

csv_docs = csv_loader.load()

print("문서의 수:", len(csv_docs))
print("-" * 50)
print("처음 문서의 메타데이터: \n", csv_docs[0].metadata)
print("-" * 50)
print("처음 문서의 내용: \n", csv_docs[0].page_content)

문서의 수: 10
--------------------------------------------------
처음 문서의 메타데이터: 
 {'source': 'KIA 타이거즈', 'row': 0}
--------------------------------------------------
처음 문서의 내용: 
 Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


### 3. **PDF 파일 로더**

- uv add langchain_community pypdf

In [10]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화 (근로기준법 문서)
pdf_loader = PyPDFLoader('./data/labor_law.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

PDF 문서 개수: 20


# 텍스트 분할(Text Splitting) 

- 대규모 텍스트 문서를 처리할 때 매우 중요한 전처리 단계
- 고려사항:
    1. 문서의 구조와 형식
    2. 원하는 청크 크기
    3. 문맥 보존의 중요도
    4. 처리 속도 

In [11]:
print(f'첫 번째 문서: {pdf_docs[0]}')

첫 번째 문서: page_content='법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖

In [15]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

첫 번째 문서의 텍스트 길이: 1811


### 1. **CharacterTextSplitter**
- 가장 기본적인 분할 방식
- 문자 수를 기준으로 텍스트를 분할
- 단순하지만 문맥을 고려하지 않는다는 단점이 있음

- 설치: pip install langchain_text_splitters 또는 uv add langchain_text_splitters

In [16]:
from langchain_text_splitters import CharacterTextSplitter 

# 텍스트 분할기 초기화 (기본 설정값 적용 )
text_splitter = CharacterTextSplitter(

    # CharacterTextSplitter의 기본 설정값
    separator = "\n\n",         # 청크 구분자: 두 개의 개행문자
    is_separator_regex = False,  # 구분자가 정규식인지 여부

    # TextSplitter의 기본 설정값
    chunk_size = 1000,          # 청크 길이
    chunk_overlap = 200,        # 청크 중첩
    length_function = len,      # 길이 함수 (문자열 길이)
    keep_separator = False,     # 구분자 유지 여부
    add_start_index = False,   # 시작 인덱스 추가 여부
    strip_whitespace = True,   # 공백 제거 여부
)

# 텍스트 분할 - split_text() 메서드 사용
texts = text_splitter.split_text(long_text)

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(texts)}')

# 첫 번째 분할된 텍스트 출력
print(f'첫 번째 분할된 텍스트: {texts[0]}')

분할된 텍스트 개수: 1
첫 번째 분할된 텍스트: 법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급

In [17]:
from langchain_text_splitters import CharacterTextSplitter

# 문장 구분자를 개행문자로 설정
text_splitter = CharacterTextSplitter(
    separator = "\n",        # 청크 구분자: 개행문자
    chunk_size = 1000,       # 청크 길이
    chunk_overlap = 200      # 청크 중첩
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])   # 첫 번째 문서만 분할

# 분할된 텍스트 개수 출력
print(f'분할된 텍스트 개수: {len(chunks)}')

# 각 청크의 텍스트 길이 출력
for i, chunk in enumerate(chunks):
    print(f'청크 {i+1}의 텍스트 길이: {len(chunk.page_content)}')

# 첫 번째 청크의 텍스트 출력
print(f'첫 번째 청크의 텍스트: {chunks[0].page_content}')

분할된 텍스트 개수: 3
청크 1의 텍스트 길이: 936
청크 2의 텍스트 길이: 944
청크 3의 텍스트 길이: 273
첫 번째 청크의 텍스트: 법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 

### 2. **RecursiveCharacterTextSplitter**

- 재귀적으로 텍스트를 분할
- 구분자를 순차적으로 적용하여 큰 청크에서 시작하여 점진적으로 더 작은 단위로 분할
- 문맥을 더 잘 보존할 수 있음  


In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,             # 청크 크기  
    chunk_overlap=200,           # 청크 중 중복되는 부분 크기
    length_function=len,         # 글자 수를 기준으로 분할
    separators=["\n\n", "\n", " ", ""],  # 구분자 - 재귀적으로 순차적으로 적용 
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("-" * 100)
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 56
각 청크의 길이: [936, 944, 273, 941, 942, 941, 963, 637, 971, 991, 420, 946, 985, 563, 953, 980, 341, 961, 997, 661, 996, 957, 336, 951, 987, 367, 966, 959, 697, 981, 967, 950, 959, 842, 990, 975, 440, 966, 970, 448, 947, 967, 413, 939, 997, 393, 952, 958, 255, 987, 687, 997, 975, 377, 978, 888]

법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 
----------------------------------------------------------------------------------------------------
 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.

2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 

### 3. **정규표현식 사용**

In [20]:
from langchain_text_splitters import CharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = CharacterTextSplitter(
    separator=r'(?<=[.!?])\s+',  # 각 Document 객체의 page_content 속성을 문장으로 분할
    chunk_size=1000,
    chunk_overlap=200,
    is_separator_regex=True,      # 구분자가 정규식인지 여부: True
    keep_separator=True,          # 구분자 유지 여부: True
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs)  # 모든 문서를 분할
print(f"생성된 텍스트 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

# 각 청크의 시작 부분과 끝 부분 확인 - 5개 청크만 출력
for chunk in chunks[:5]:
    print(chunk.page_content[:200])
    print("...")
    print(chunk.page_content[-200:])
    print("=" * 100)
    print()

생성된 텍스트 청크 수: 55
각 청크의 길이: [939, 944, 273, 937, 922, 941, 986, 561, 971, 973, 387, 946, 751, 764, 935, 957, 401, 826, 890, 830, 996, 998, 951, 985, 274, 961, 889, 776, 947, 926, 999, 996, 479, 929, 890, 594, 961, 963, 409, 947, 998, 329, 866, 954, 464, 955, 989, 255, 999, 670, 923, 912, 291, 981, 880]

법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 
...
이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5.

2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게
...
등한 지위에서 자유의사에 따라 결정하여야 한다.
 
제5조(근로조건의 준수) 근로자와 사용자는 각자가 단체협약, 취업규칙과 근로계약을 지키고 성실하게 

`(4) 유사도 기반 검색`

### 4. **토큰 수를 기반으로 분할**

`(1) tiktoken`  
- OpenAI에서 만든 BPE Tokenizer

In [21]:
# 첫번째 문서 객체의 텍스트 길이
len(pdf_docs[0].page_content)

1811

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# TikToken 인코더를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", 
    # model_name="gpt-4.1-mini",
    chunk_size=300, 
    chunk_overlap=0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]])  # 첫 번째 문서만 분할

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(chunk.page_content[:50])
    print("-" * 50)
    print(chunk.page_content[-50:])
    print("=" * 50)
    print()

생성된 청크 수: 6
각 청크의 길이: [423, 280, 301, 240, 287, 273]
법제처                                               
--------------------------------------------------
202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973

고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고
--------------------------------------------------
 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>

1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는
--------------------------------------------------
대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다.

6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에
--------------------------------------------------
139조제1항에 따른 근로시간의 범위에
서 근로자와 사용자 사이에 정한 근로시간을 말한다.

9. “단시간근로자”란 1주 동안의 소정근로시간이 그 사업장에서 같은 종류의 업무에 종사하
--------------------------------------------------
건의 결정) 근로조건은 근로자와 사용자가 동등한 지위에서 자유의사에 따라 결정하여야 한다.



In [23]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")
# tokenizer = tiktoken.encoding_for_model("gpt-4.1-mini")

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = [tokenizer.decode([token]) for token in tokens[:10]]
    print(token_strings)

    print("=" * 50)
    print()

297
[28617, 243, 38187, 36155, 246, 12216, 220, 16, 26857, 8790]
['�', '�', '제', '�', '�', '                                                           ', ' ', '1', '                                                      ', ' �']

259
[35495, 27797, 75265, 116, 58189, 64189, 320, 94801, 18202, 230]
['고', '용', '�', '�', '동', '부', ' (', '임', '�', '�']

290
[16, 13, 1054, 90161, 120, 17835, 26799, 863, 39519, 222]
['1', '.', ' “', '�', '�', '로', '자', '”', '�', '�']

239
[21, 13, 1054, 169, 237, 231, 90161, 254, 94801, 18202]
['6', '.', ' “', '�', '�', '�', '�', '�', '임', '�']

286
[24, 13, 1054, 9019, 101, 30426, 63375, 90161, 120, 17835]
['9', '.', ' “', '�', '�', '시', '간', '�', '�', '로']



`(2) Hugging Face 토크나이저`  
- Hugging Face tokenizer 모델의 토큰 수를 기준으로 분할
- uv add langchain_huggingface sentence_transformers

In [25]:
from transformers import AutoTokenizer 
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

tokenizer

/Users/dongjun/Desktop/Project/ms-ai-edu/ktds-llm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


XLMRobertaTokenizerFast(name_or_path='BAAI/bge-m3', vocab_size=250002, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)

In [26]:
# 토크나이저 인코딩 - 문장을 토큰(ID)으로 변환
tokens = tokenizer.encode("안녕하세요. 반갑습니다.")
print(tokens)

[0, 107687, 5, 20451, 54272, 16367, 5, 2]


In [27]:
# 토큰을 출력 (토큰 ID를 실제 토큰(문자열)로 변환)
print(tokenizer.convert_ids_to_tokens(tokens)) 

['<s>', '▁안녕하세요', '.', '▁반', '갑', '습니다', '.', '</s>']


In [28]:
# 디코딩 - 토큰을 문자열로 변환
print(tokenizer.decode(tokens, skip_special_tokens=True))

안녕하세요. 반갑습니다.


In [30]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Huggingface 토크나이저를 사용하여 재귀적 텍스트 분할기 초기화
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300, 
    chunk_overlap=0,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents([pdf_docs[0]]) # 첫 번째 문서만 분할

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 토큰화 결과 확인 (첫 10개 토큰만 출력)
    print(tokens[:10])
    # 토큰 ID를 실제 토큰(문자열)로 변환해서 출력
    token_strings = tokenizer.convert_ids_to_tokens(tokens[:10]) 
    print(token_strings)

    print("=" * 50)
    print()

생성된 청크 수: 4
각 청크의 길이: [620, 527, 453, 204]

284
[0, 37668, 4654, 24247, 106, 40343, 10629, 25547, 26444, 24740]
['<s>', '▁법', '제', '처', '▁1', '▁국가', '법', '령', '정보', '센터']

264
[0, 6600, 304, 6740, 132, 2905, 367, 16, 106, 1504]
['<s>', '▁제', '2', '조', '(', '정', '의', ')', '▁1', '▁이']

267
[0, 3217, 52, 3999, 2905, 132, 1493, 3779, 16, 23147]
['<s>', '▁8.', '▁“', '소', '정', '(', '所', '定', ')', '근']

124
[0, 6600, 910, 6740, 132, 83654, 19123, 993, 101840, 6775]
['<s>', '▁제', '6', '조', '(', '균', '등', '한', '▁처', '우']



### 5. **Semantic Chunking**

- **SemanticChunker**는 텍스트를 의미 단위로 **분할**하는 특수한 텍스트 분할도구 

- 단순 길이 기반이 아닌 **의미 기반**으로 텍스트를 청크화하는데 효과적

- **breakpoint_threshold_type**: Text Splitting의 다양한 임계값(Threshold) 설정 방식 (통계적 기법) 

    - **Gradient** 방식: 임베딩 벡터 간의 **기울기 변화**를 기준으로 텍스트를 분할
    - **Percentile** 방식: 임베딩 거리의 **백분위수**를 기준으로 분할 지점을 결정
    - **Standard Deviation** 방식: 임베딩 거리의 **표준편차**를 활용하여 유의미한 변화점을 찾아서 분할
    - **Interquartile** 방식: 임베딩 거리의 **사분위수 범위**를 기준으로 이상치를 감지하여 분할

- 설치: pip install langchain_experimental 또는 uv add langchain_experimental


In [32]:
from langchain_experimental.text_splitter import SemanticChunker 
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델을 사용하여 SemanticChunker를 초기화 
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 임계값 타입 설정 (gradient, percentile, standard_deviation, interquartile)
)

In [33]:
chunks = text_splitter.split_documents([pdf_docs[0]])

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")
print()

tokenizer = tiktoken.get_encoding("cl100k_base")

for chunk in chunks[:5]:

    # 각 청크를 토큰화
    tokens = tokenizer.encode(chunk.page_content)
    # 각 청크의 단어 수 확인
    print(len(tokens))
    # 각 청크의 내용을 확인
    print(chunk.page_content[:100])
    print("=" * 50)
    print()

생성된 청크 수: 3
각 청크의 길이: [151, 542, 1104]



NameError: name 'tiktoken' is not defined

# 문서 임베딩(Document Embedding)

- 개념: 
    - 텍스트를 벡터(숫자 배열)로 변환하는 과정
    - 문서의 의미적 특성을 수치화하여 컴퓨터가 이해하고 처리할 수 있는 형태로 변환 

- 목적:
    - 텍스트 간 유사도 계산 가능
    - 벡터 데이터베이스 저장 및 검색
    - 의미 기반 문서 검색 구현

- LangChain의 임베딩 모델 종류:
    - OpenAI 임베딩
    - HuggingFace 임베딩 

### 1. **OpenAI**

- LangChain에서 가장 널리 사용되는 임베딩 모델 중 하나

- 주요 특징:
    1. 고품질의 임베딩 생성
    2. 다양한 언어 지원 (다국어 지원)
    3. 일관된 성능
    4. 손쉬운 통합

- 사용시 주의사항:
    1. API 키 설정이 필요 (환경 변수 OPENAI_API_KEY)
    2. API 사용량에 따른 비용 발생
    3. 긴 텍스트는 자동으로 분할되지 않으므로 필요시 TextSplitter를 사용


- 모델별 특징
    ```
    모델                    페이지/달러    MTEB 성능     최대입력
    text-embedding-3-small  62,500       62.3%       8191
    text-embedding-3-large   9,615       64.6%       8191
    text-embedding-ada-002  12,500       61.0%       8191
    ```

- 임베딩 벡터 특성
    1. small: 1536 차원
    1. large: 3072 차원
    1. dimensions 파라미터로 차원 축소 가능

`(1) embedding 모델`

In [18]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-large",  # 사용할 모델 이름
    dimensions=None, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 객체 출력
embeddings_model

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x1315bdfd0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x1315be900>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [34]:
# 임베딩 모델의 컨텍스트 길이 확인
embeddings_model.embedding_ctx_length

8191

In [35]:
from langchain_openai import OpenAIEmbeddings

# OpenAIEmbeddings 모델 생성할 때 임베딩 차원을 지정하는 예시
embeddings_openai = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    dimensions=1024, # 원하는 임베딩 차원 수를 지정 가능 (기본값: None)
    )

# 임베딩 모델의 임베딩 차원 확인 
embeddings_openai.dimensions

1024

`(2) embed_documents 사용`

In [36]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_openai = embeddings_openai.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_openai)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_openai[0])}")
print(document_embeddings_openai[0])

임베딩 벡터의 개수: 5
임베딩 벡터의 차원: 1024
[-0.0026579881086945534, 0.013920078054070473, -0.0028032048139721155, 0.01730155758559704, 0.021222414448857307, -0.05252702906727791, -0.0035889321006834507, 0.05402068793773651, -0.020983843132853508, -0.03667763993144035, 0.007878017611801624, 0.008012861013412476, -0.021388376131653786, -0.03122163563966751, 0.005627155303955078, -0.01792391575872898, -0.047797106206417084, 0.011565489694476128, 0.05933148041367531, -0.052775971591472626, -0.016845161095261574, -0.03192697465419769, -0.0215750839561224, -0.02611829712986946, 0.0050203558057546616, -0.045805562287569046, 0.057879310101270676, 0.02038222923874855, 0.000326413894072175, -0.026844382286071777, 0.056344158947467804, -0.01757124625146389, -0.0344371534883976, -0.07086584717035294, 0.023525139316916466, 0.040702223777770996, 0.0029743534978479147, -0.007831340655684471, -0.005902030039578676, 0.028504004701972008, 0.008702641353011131, 0.031678032130002975, -0.002951015019789338, 0.02989393

`(3) embed_query 사용`

In [22]:
embedded_query_openai = embeddings_openai.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query_openai)}")
print(embedded_query_openai)

쿼리 임베딩 벡터의 차원: 1024
[-0.026167644187808037, 0.025727111846208572, 0.0005699392058886588, 0.006574951112270355, 0.014141100458800793, -0.05198286101222038, -0.030484864488244057, 0.041542235761880875, -0.0030672091525048018, 0.017169762402772903, -0.0024366965517401695, 0.001000146963633597, -0.005644325632601976, -0.08537524193525314, 0.010462651960551739, -0.01789664290845394, -0.069383904337883, -0.026035483926534653, 0.026013458147644997, -0.08030911535024643, -0.03389899432659149, 0.026960602030158043, -0.04275370016694069, 0.004777026828378439, 0.012929635122418404, -0.061057835817337036, 0.012445049360394478, -3.5900837247027084e-05, -0.0125331562012434, -0.03955983743071556, 0.02933947928249836, -0.02193853072822094, -0.0017401042860001326, -0.06841473281383514, 0.05788600072264671, -0.0039620413444936275, -0.0032021221704781055, -0.009708239696919918, -0.005333199165761471, 0.02645399048924446, -0.013788674026727676, 0.044053271412849426, 0.00382712809368968, 0.0408373847603797

`(4) 유사도 기반 검색`

In [37]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

# 쿼리와 가장 유사한 문서 찾기 함수
def find_most_similar(
        query: str, 
        doc_embeddings: np.ndarray,
        embeddings_model=OpenAIEmbeddings(model="text-embedding-3-small")
        ) -> tuple[str, float]:
    
    # 쿼리 임베딩: OpenAI 임베딩 사용 
    query_embedding = embeddings_model.embed_query(query)

    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # 가장 유사한 문서 인덱스 찾기
    most_similar_idx = np.argmax(similarities)

    # 가장 유사한 문서와 유사도 반환: 문서, 유사도
    return documents[most_similar_idx], similarities[most_similar_idx]

# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(
        query, 
        document_embeddings_openai, 
        embeddings_model=embeddings_openai
        )
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

쿼리: 인공지능이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도: 0.7147

쿼리: 딥러닝과 머신러닝의 관계는 어떻게 되나요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도: 0.6752

쿼리: 컴퓨터가 이미지를 이해하는 방법은?
가장 유사한 문서: 컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다.
유사도: 0.7050



### 2. **Huggingface**

- LangChain에서 오픈소스 기반의 대표적인 임베딩 모델

- 주요 특징:
    1. 로컬 환경에서 실행 가능
    2. 다양한 사전학습 모델 지원
    3. 커스텀 모델 학습 및 적용 가능
    4. 무료 사용 가능 (API 비용 없음)

- 사용시 주의사항:
    1. 로컬 컴퓨팅 자원 필요 (CPU/GPU)
    2. 초기 모델 다운로드 시간 소요
    3. 메모리 사용량 고려 필요
    4. transformers 라이브러리 설치 필요

- 대표적인 임베딩 모델:
    ```
    모델                            차원      언어             특징      
    all-MiniLM-L6-v2               384     다국어     빠른 속도, 적은 메모리
    all-mpnet-base-v2              768     다국어     높은 성능, 중간 크기
    multilingual-e5-large         1024     다국어     최고 성능, 큰 메모리
    ```

- 임베딩 벡터 특성:
    1. 모델별로 다양한 차원 제공 (128 ~ 1024)
    2. sentence-transformers 기반 구현
    3. BERT 계열 모델 구조 사용
    4. 코사인 유사도 기반 검색 최적화

`(1) embedding 모델`

- langchain_huggingface 설치 필요

In [38]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  

# Hugging Face의 임베딩 모델 생성
embeddings_bgem3 = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",          # 사용할 모델 이름 - BAAI BGE-m3 모델 (한국어 성능 우수)
    # model_kwargs={'device': 'cuda'}  # GPU 사용시
    # model_kwargs={'device': 'mps'}   # Mac M1 사용시
)

# 임베딩 객체 출력
embeddings_bgem3

KeyboardInterrupt: 

`(2) embed_documents 사용`

In [ ]:
# 문서 임베딩
document_embeddings_bgem3 = embeddings_bgem3.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_bgem3)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_bgem3[0])}")
print(document_embeddings_bgem3[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_bgem3.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_bgem3, embeddings_model=embeddings_bgem3) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

### 3. **Ollama**

- LangChain에서 로컬 실행에 최적화된 경량 임베딩 모델

- 주요 특징:
    1. 완전한 로컬 실행 환경 제공
    2. 빠른 추론 속도
    3. 간단한 설치 및 실행 과정
    4. Docker 기반 손쉬운 배포

- 사용시 주의사항:
    1. Ollama 서버 실행 필요
    2. 모델별 시스템 요구사항 확인
    3. API 엔드포인트 설정 필요
    4. 동시 요청 처리량 고려

- 대표적인 임베딩 모델:
    ```
    모델                차원       언어        특징
    llama2              4096      영어       범용성 높은 기본 모델
    nomic-embed-text    768       영어       경량화된 고성능 모델
    codellama          2048       다국어     코드 특화 임베딩
    ```

- 임베딩 벡터 특성:
    1. 모델별 고정 차원 사용
    2. 최적화된 양자화 지원
    3. 배치 처리 최적화

`(1) embedding 모델`

- langchain_ollama 설치 필요

In [ ]:
from langchain_ollama import OllamaEmbeddings 

# OllamaEmbeddings 모델 생성
# embeddings_ollama = OllamaEmbeddings(
#     model="nomic-embed-text",          # 사용할 모델 이름
#     base_url="http://localhost:11434"  # Ollama 서버 주소
# )
embeddings_ollama = OllamaEmbeddings(model="bge-m3")

# 임베딩 객체 출력
embeddings_ollama

`(2) embed_documents 사용`

In [ ]:
# 문서 컬렉션
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하고 생성하는 기술입니다.",
    "컴퓨터 비전은 컴퓨터가 디지털 이미지나 비디오를 이해하는 방법을 연구합니다."
]

# 문서 임베딩
document_embeddings_ollama = embeddings_ollama.embed_documents(documents)

# 임베딩 결과 출력
print(f"임베딩 벡터의 개수: {len(document_embeddings_ollama)}")
print(f"임베딩 벡터의 차원: {len(document_embeddings_ollama[0])}")
print(document_embeddings_ollama[0])

`(3) embed_query 사용`

In [ ]:
embedded_query = embeddings_ollama.embed_query("인공지능이란 무엇인가요?")

# 쿼리 임베딩 결과 출력
print(f"쿼리 임베딩 벡터의 차원: {len(embedded_query)}")
print(embedded_query)

`(4) 유사도 기반 검색`

In [ ]:
# 예제 쿼리
queries = [
    "인공지능이란 무엇인가요?",
    "딥러닝과 머신러닝의 관계는 어떻게 되나요?",
    "컴퓨터가 이미지를 이해하는 방법은?"
]

# 각 쿼리에 대해 가장 유사한 문서 찾기
for query in queries:
    most_similar_doc, similarity = find_most_similar(query, document_embeddings_ollama, embeddings_model=embeddings_ollama) 
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {most_similar_doc}")
    print(f"유사도: {similarity:.4f}")
    print()

# 벡터 저장소 (Vector Store)

- 개념:
    - 벡터화된 데이터를 효율적으로 저장하고 검색하기 위한 특수 데이터베이스 시스템
    - 텍스트나 이미지 등의 비정형 데이터를 고차원 벡터 공간에 매핑하여 저장
    - 유사도 기반 검색을 통해 의미적으로 가까운 데이터를 빠르게 검색 가능 

- LangChain의 벡터 저장소 종류:
    - **Chroma**: 경량화된 임베딩 데이터베이스로 로컬 개발에 적합
    - **FAISS**: Facebook AI가 개발한 고성능 유사도 검색 라이브러리
    - **Pinecone**: 완전 관리형 벡터 데이터베이스 서비스
    - Milvus: 분산 벡터 데이터베이스로 대규모 데이터 처리에 적합
    - PostgreSQL: pgvector 확장을 통해 벡터 저장 및 검색 기능을 제공

- 주요 기능:
    - 벡터 색인화: 효율적인 검색을 위한 데이터 구조화를 수행
    - 근접 이웃 검색: 주어진 쿼리와 가장 유사한 벡터들을 검색 
    - 메타데이터 관리: 벡터와 관련된 부가 정보를 함께 저장하고 검색

- 사용 사례:
    - 시맨틱 문서 검색: 문서의 의미를 이해하여 검색
    - 추천 시스템: 유사한 아이템을 추천
    - 중복 데이터 감지: 유사한 콘텐츠를 검색 
    - 질의응답 시스템: 관련 문서에서 답변을 생성하는데 필요한 근거를 검색 

### **Chroma**

- 사용자 편의성이 우수한 오픈소스 벡터 저장소
- `langchain-chroma` 패키지 설치

`(1) 벡터 저장소 초기화`

In [39]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화 (근로기준법 문서)
pdf_loader = PyPDFLoader('./data/labor_law.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

len(pdf_docs)

PDF 문서 개수: 20


20

In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# TikToken 인코더를 사용하여 재귀적 텍스트 분할기 초기화 (토큰 수 기준 분할)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", 
    chunk_size=500, 
    chunk_overlap=100,
)

# split_documents() 메서드 사용 : Document 객체를 여러 개의 작은 청크 문서로 분할
chunks = text_splitter.split_documents(pdf_docs) 

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

# 각 청크의 시작 부분과 끝 부분 확인
for chunk in chunks[:5]:
    print(f"{chunk.page_content[:100]}...{chunk.page_content[-100:]}")
    print("="*100)

생성된 청크 수: 95
각 청크의 길이: [620, 476, 473, 468, 99, 627, 459, 517, 358, 607, 478, 472, 517, 387, 631, 541, 501, 474, 204, 558, 463, 459, 535, 432, 583, 497, 488, 463, 209, 609, 477, 504, 459, 512, 612, 483, 491, 476, 168, 633, 501, 471, 502, 193, 655, 509, 488, 465, 472, 637, 508, 493, 331, 589, 481, 537, 545, 496, 89, 593, 448, 473, 511, 318, 660, 528, 536, 505, 204, 624, 522, 489, 509, 129, 595, 485, 542, 530, 630, 482, 493, 379, 573, 511, 467, 196, 557, 494, 570, 550, 151, 671, 574, 480, 274]
법제처                                                            1                                    ... 제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의)...임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다.
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다.
6. “평균임금”이란 이

In [41]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름)
)

# Chroma 벡터 저장소 생성하기
chroma_db = Chroma.from_documents(  
    documents=chunks,
    embedding=embeddings_openai,    # OpenAI 임베딩 사용
    collection_name="labor_law",    # 컬렉션 이름
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

ModuleNotFoundError: No module named 'langchain_chroma'

In [ ]:
# 문서 개수 확인
chroma_db._collection.count()

`(2) 벡터 저장소 로드`  

In [ ]:
from langchain_chroma import Chroma

# 저장된 벡터 저장소를 가져오기
chroma_db = Chroma(
    collection_name="labor_law",
    embedding_function=embeddings_openai,
    persist_directory="./chroma_db",
)

In [ ]:
# 문서 개수 확인
chroma_db._collection.count()

`(3) 문서 검색`  
- 유사도 검색
    - 주어진 쿼리와 가장 유사한 문서를 반환
    -  k=5는 상위 5개의 결과를 반환하도록 지정
    - filter를 사용하여 특정 출처의 문서만 검색 가능

In [ ]:
query = "'탄력 근로에 대해서 설명해주세요"
results = chroma_db.similarity_search(
    query,
    k=5,
    filter={"source": "./data/labor_law.pdf"}
)

print("유사도 검색 결과:")
for doc in results:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("=" * 100)

- 유사도 점수가 포함된 검색
    - 유사도 점수를 함께 반환
    - 점수가 낮을수록 더 유사한 것을 의미 (거리 기준으로 점수가 산정되기 때문)

In [ ]:
query = "'탄력 근로에 대해서 설명해주세요"
results = chroma_db.similarity_search_with_score(
    query,
    k=5,
)

print("점수가 포함된 유사도 검색 결과:\n")
for doc, score in results:
    print(f"- 점수: {score:.4f}")
    print(f"  내용: {doc.page_content}")
    print(f" 메타데이터: {doc.metadata}")
    print("=" * 100)

# 벡터 저장소 기반 RAG 검색기 (Retriever)



`(1) Top K`

In [ ]:
retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "탄력 근로에 대해서 설명해주세요"
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-" * 100)

`(2) 임계값 지정`
- Similarity score threshold (기준 스코어 이상인 문서를 대상으로 추출)

In [ ]:
from langchain_community.utils.math import cosine_similarity

retriever = chroma_db.as_retriever(
    search_type='similarity_score_threshold',       # cosine 유사도
    search_kwargs={'score_threshold': 0.3, 'k':5},  # 0.3 이상인 문서를 추출
)

query = "탄력 근로에 대해서 설명해주세요"

# 쿼리와 유사한 문서 검색
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_openai.embed_query(query)], 
        [embeddings_openai.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content}\n[유사도: {score}]")
    print("-" * 100)

`(3) MMR(Maximal Marginal Relevance) 검색`

In [ ]:
# MMR - 다양성 고려 (lambda_mult 작을수록 더 다양하게 추출)
retriever = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 3,                 # 검색할 문서의 수
        'fetch_k': 8,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.5,     # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)


query = "탄력 근로에 대해서 설명해주세요"

# 쿼리와 유사한 문서 검색
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    score = cosine_similarity(
        [embeddings_openai.embed_query(query)], 
        [embeddings_openai.embed_query(doc.page_content)]
        )[0][0]
    print(f"-{i}-\n{doc.page_content}\n[유사도: {score}]")
    print("-" * 100)

# Naive RAG 구현 

`(1) 벡터 저장소 로드`
- `Chroma` 벡터 저장소를 로드

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 생성
embeddings_openai = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름
    )

# 저장된 벡터 저장소를 가져오기
chroma_db = Chroma(
    collection_name="labor_law",
    embedding_function=embeddings_openai,
    persist_directory="./chroma_db",
)

# 문서 개수 확인
print(f"저장된 문서 수: {chroma_db._collection.count()}")

`(2) 검색기(Retriever) 초기화`

- mmr 검색을 사용하는 Retriever 사용
- 다양성을 높이는 설정을 사용

In [ ]:
# mmr 검색기 생성
retriever = chroma_db.as_retriever(
    search_type='mmr',
    search_kwargs={
        'k': 5,                  # 검색할 문서의 수
        'fetch_k': 10,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
        'lambda_mult': 0.3,      # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
        },
)

In [ ]:
# 검색 테스트 
query = "탄력 근로에 대해서 설명해주세요"

# 쿼리와 유사한 문서 검색
retrieved_docs = retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"-{i}-\n{doc.page_content}\n[출처: {doc.metadata['source']}]")
    print("-" * 100)

`(3) RAG 프롬프트 구성`

- 작성 기준: 
    - LangChain의 ChatPromptTemplate 클래스 사용
    - 변수 처리는 {context}, {question} 형식 사용
    - 답변은 한글로 출력되도록 프롬프트 작성
    
- 아래 템플릿 코드를 기반으로 다음 내용을 참고하여 작성합니다. 

    1. 프롬프트 구성요소:
        - 작업 지침
        - 컨텍스트 영역
        - 질문 영역
        - 답변 형식 가이드

    2. 작업 지침:
        - 컨텍스트 기반 답변 원칙
        - 외부 지식 사용 제한
        - 불확실성 처리 방법
        - 답변 불가능한 경우의 처리 방법

    3. 답변 형식:
        - 핵심 답변 섹션
        - 근거 제시 섹션
        - 추가 설명 섹션 (필요시)

    4. 제약사항 반영:
        - 답변은 사실에 기반해야 함
        - 추측이나 가정을 최소화해야 함
        - 명확한 근거 제시가 필요함
        - 구조화된 형태로 작성되어야 함

In [ ]:
# Prompt 템플릿 (기본 예시)
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿 출력
prompt.pretty_print()

In [ ]:
# Prompt 템플릿 (커스텀 예시)
from langchain_core.prompts import ChatPromptTemplate

template = """주어진 컨텍스트를 기반으로 질문에 답변하시오.

[지침]
- 컨텍스트에 있는 정보만을 사용하여 답변할 것
- 외부 지식이나 정보를 사용하지 말 것
- 컨텍스트에서 답을 찾을 수 없는 경우 "주어진 정보만으로는 답변하기 어렵습니다."라고 응답할 것
- 불확실한 경우 명확히 그 불확실성을 표현할 것
- 답변은 논리적이고 구조화된 형태로 제공할 것
- 답변은 한국어를 사용할 것 

[컨텍스트]
{context}

[질문]
{question}

[답변 형식]
1. 핵심 답변: (질문에 대한 직접적인 답변)
2. 근거: (컨텍스트에서 발견된 관련 정보)
3. 추가 설명: (필요한 경우 부연 설명 제공)

[답변]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿 출력
prompt.pretty_print()

`(4) RAG 체인 구성`
- LangChain의 LCEL 문법을 사용
- 검색 결과를 프롬프트의 'context'로 전달하고,
- 사용자가 입력한 질문을 그래도 프롬프트의 'question'에 전달
- LLM 설정:
    - ChatOpenAI 사용 ('gpt-4.1-mini' 모델)
    - temperature: 답변의 일관성을 가져가는 설정값을 사용 
    - 기타 필요한 설정 
- 출력 파서: 문자열 부분만 출력되도록 구성 

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.7,
    top_p=0.9,
)

# 문서 포맷팅
def format_docs(docs):
    return "\n\n".join([f"{doc.page_content}" for doc in docs])

# RAG 체인 생성
rag_chain = (
    RunnableParallel(
        {
            "context": retriever | format_docs, 
            "question": RunnablePassthrough()
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "탄력 근로에 대해서 설명해주세요"
output = rag_chain.invoke(query)

print(f"쿼리: {query}")
print("답변:")
print(output)

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_screenshot_0001.png)

`(5) LangServe 서버 구성`
- app/rag.py 파일을 생성하여 RAG 체인과 관련된 코드를 작성함.

    ```python
    # app/rag.py
    from dotenv import load_dotenv

    from langchain_core.runnables import RunnablePassthrough
    from langchain_core.output_parsers import StrOutputParser
    from langchain_openai import ChatOpenAI, OpenAIEmbeddings
    from langchain_chroma import Chroma
    from langchain_core.prompts import ChatPromptTemplate

    # 환경변수 로드
    load_dotenv()


    ######################
    #  RAG 체인 구성
    ######################

    # OpenAI 임베딩 모델 생성
    embeddings_openai = OpenAIEmbeddings(
        model="text-embedding-3-small",  # 사용할 모델 이름
        )

    # 저장된 벡터 저장소를 가져오기
    chroma_db = Chroma(
        collection_name="labor_law",
        embedding_function=embeddings_openai,
        persist_directory="./chroma_db",
    )

    print("Chroma DB loaded")
    print(chroma_db._collection.count())  # 벡터 저장소에 있는 문서 수 출력

    # 검색기 초기화å
    retriever = chroma_db.as_retriever(
        search_type='mmr',
        search_kwargs={
            'k': 5,                  # 검색할 문서의 수
            'fetch_k': 10,           # mmr 알고리즘에 전달할 문서의 수 (fetch_k > k)
            'lambda_mult': 0.3,      # 다양성을 고려하는 정도 (1은 최소 다양성, 0은 최대 다양성을 의미. 기본값은 0.5)
            },
    )

    # Prompt 템플릿 생성
    template = """주어진 컨텍스트를 기반으로 질문에 답변하시오.

    [지침]
    - 컨텍스트에 있는 정보만을 사용하여 답변할 것
    - 외부 지식이나 정보를 사용하지 말 것
    - 컨텍스트에서 답을 찾을 수 없는 경우 "주어진 정보만으로는 답변하기 어렵습니다."라고 응답할 것
    - 불확실한 경우 명확히 그 불확실성을 표현할 것
    - 답변은 논리적이고 구조화된 형태로 제공할 것
    - 답변은 한국어를 사용할 것 

    [컨텍스트]
    {context}

    [질문]
    {question}

    [답변 형식]
    1. 핵심 답변: (질문에 대한 직접적인 답변)
    2. 근거: (컨텍스트에서 발견된 관련 정보)
    3. 추가 설명: (필요한 경우 부연 설명 제공)

    [답변]
    """

    prompt = ChatPromptTemplate.from_template(template)

    # LLM 설정
    llm = ChatOpenAI(
        model="gpt-4.1-mini",
        temperature=0.7,
        top_p=0.9,
    )

    # 문서 포맷팅
    def format_docs(docs):
        return "\n\n".join([f"{doc.page_content}" for doc in docs])

    # RAG 체인 생성

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    ```


- server.py 파일에 새로운 엔드포인트를 추가하여 RAG 체인을 호출하도록 설정함.

    ```python
    # app/server.py
    from fastapi import FastAPI
    from dotenv import load_dotenv
    from app.rag import rag_chain
    from langchain_openai import ChatOpenAI
    from langserve import add_routes

    # 환경변수 로드
    load_dotenv()

    # FastAPI 서버를 설정
    app = FastAPI(
        title="LangChain Server",
        version="1.0",
        description="Spin up a simple api server using Langchain's Runnable interfaces",
    )

    # 라우팅 설정
    add_routes(
        app,
        ChatOpenAI(model="gpt-4.1-mini"),
        path="/openai",   # OpenAI 모델에 대한 경로
    )

    add_routes(
        app,
        rag_chain,
        path="/rag",  # RAG 체인에 대한 경로
    )

    # FastAPI 서버 실행
    if __name__ == "__main__":
        import uvicorn

        uvicorn.run(app, host="localhost", port=8000)
    ```

![LangServe Screenshot](https://raw.githubusercontent.com/tsdata/image_files/main/202505/rag_screenshot_0002.png)